In [100]:
import os
import sys
import pandas as pd
from data_helper import create_none_existing_folder
from data_helper import get_csv_dataframes_from_folder

#### process commitments 

#### update state name in same format


In [101]:
input_csv_file_path = '../data/commitments/gold/commitments.csv'

In [102]:
commitments_df = pd.read_csv(input_csv_file_path)

In [104]:
commitments_df.head()

,name,commitment_year,state,high_school,club,position,division,college,gender,club_rank,weight,club_year,height,web_source,profile_link
0,Isabella Sabatino,2022.0,MD,Towson,SkyWalkers,M,D3,Pomona Pitzer,girl,NaN,NaN,NaN,NaN,laxnumbers,NaN
1,Natalie Clere,2022.0,NY,Onondaga,NaN,M,D2,Lindenwood,girl,NaN,NaN,NaN,NaN,laxnumbers,NaN
2,Claire Ashburn,2022.0,MD,Harford,NaN,A,D1,Winthrop,girl,NaN,NaN,NaN,NaN,laxnumbers,NaN
3,Gabby Trzeciak,2022.0,MD,Harford,Looneys,A,D2,Lock Haven,girl,NaN,NaN,NaN,NaN,laxnumbers,NaN
4,Trysten Burns,2022.0,NY,Onondaga,Haudenosaunee,A,D2,Newberry,girl,NaN,NaN,NaN,NaN,laxnumbers,NaN


In [105]:
commitments_df.shape

(49758, 15)

In [106]:
# define method to get abrivatation for input name
get_abv=lambda z: ''.join([str(x[0]).upper() for x in z.split(' ')]) if type(z)==str and len(z.split(' '))>1 else z

In [107]:
commitments_df['state']=commitments_df['state'].map(get_abv)

In [108]:
commitments_df['state'][2050:2060]

2050    PA
2051    PA
2052    NY
2053    NY
2054    MD
2055    NY
2056    PA
2057    MD
2058    VA
2059    GA
Name: state, dtype: object

In [109]:
commitments_df['state'].value_counts()

NY     6809
MD     3770
PA     3514
NJ     3371
MA     2631
       ... 
Co        1
NB        1
HA        1
PNW       1
pa        1
Name: state, Length: 109, dtype: int64

#### set commitment_year values

In [110]:
commitment_year_outliers=[20225.0,2026.0]
commitments_df=commitments_df.drop(commitments_df[commitments_df['commitment_year'].isin(commitment_year_outliers)].index)
commitments_df = commitments_df[commitments_df['commitment_year'].notna()]
commitments_df['commitment_year']=commitments_df['commitment_year'].astype(int)
print(commitments_df.shape)
commitments_df.head()

(49469, 15)


,name,commitment_year,state,high_school,club,position,division,college,gender,club_rank,weight,club_year,height,web_source,profile_link
0,Isabella Sabatino,2022,MD,Towson,SkyWalkers,M,D3,Pomona Pitzer,girl,NaN,NaN,NaN,NaN,laxnumbers,NaN
1,Natalie Clere,2022,NY,Onondaga,NaN,M,D2,Lindenwood,girl,NaN,NaN,NaN,NaN,laxnumbers,NaN
2,Claire Ashburn,2022,MD,Harford,NaN,A,D1,Winthrop,girl,NaN,NaN,NaN,NaN,laxnumbers,NaN
3,Gabby Trzeciak,2022,MD,Harford,Looneys,A,D2,Lock Haven,girl,NaN,NaN,NaN,NaN,laxnumbers,NaN
4,Trysten Burns,2022,NY,Onondaga,Haudenosaunee,A,D2,Newberry,girl,NaN,NaN,NaN,NaN,laxnumbers,NaN


In [111]:
commitments_df['division'].value_counts()

D1            14604
D3            13081
D2             7865
NCAA DI        1685
NAIA           1621
NCAA DIII      1469
Club           1333
DI              889
NCAA DII        545
JUCO            347
DIII             78
DII              75
MCLA DI          22
MCLA DII         11
MCLA              7
WCLA – DI         6
WCLA – DII        6
NJCAA             2
CUFLA             2
NAIA DI           1
Name: division, dtype: int64

we need to merge data 
NCAA DI and  DI   to      D1,
NCAA DII and DII  to      D2
D3 NCAA and DIII to      D3

In [112]:
division_map_dict = {
    'NCAA DI':'D1',
    'NCAA DII':'D2',
    'NCAA DIII':'D3',
    'D3 NCAA':'D3',
    'DI':'D1',
    'DII':'D2',
    'DIII':'D3'
}

In [113]:
def map_division_on_string(input_string:str,map:dict):
    for key, val in map.items():
        if key.replace(' ','').lower() == str(input_string).replace(' ','').lower():
            return val
    return input_string
    


In [114]:
def map_postition_on_string(input_string:str,map:dict):
    for key, val in map.items():
        if key.lower() in str(input_string).lower():
            return val
    return input_string
    


In [115]:
position_map_dict = {
    'Midfield':'M',
    'Middie':'M',
    'Midflied':'M',
    'Midfiled':'M',
    'Defense':'D',
    'Attack':'A',
    'Goal':'G',
    'SSMF':'M',
    'FO/DRAW':'FO',
    'Face Off Specialist':'FO',
    'FOGO':'FO',
    'Faceoff':'FO',
    'Draw':'DRAW'
    }

In [116]:
commitments_df['position']=commitments_df['position'].apply(lambda x: map_postition_on_string(x,position_map_dict))
commitments_df['position'].value_counts()

M       16953
A       12584
D       11594
G        4809
LSM      1353
FO       1325
DRAW        8
Name: position, dtype: int64

In [117]:
commitments_df['division'].value_counts()

D1            14604
D3            13081
D2             7865
NCAA DI        1685
NAIA           1621
NCAA DIII      1469
Club           1333
DI              889
NCAA DII        545
JUCO            347
DIII             78
DII              75
MCLA DI          22
MCLA DII         11
MCLA              7
WCLA – DI         6
WCLA – DII        6
NJCAA             2
CUFLA             2
NAIA DI           1
Name: division, dtype: int64

In [118]:
commitments_df['division']=commitments_df['division'].apply(lambda x: map_division_on_string(x,division_map_dict))
commitments_df['division'].value_counts()

D1            17178
D3            14628
D2             8485
NAIA           1621
Club           1333
JUCO            347
MCLA DI          22
MCLA DII         11
MCLA              7
WCLA – DI         6
WCLA – DII        6
NJCAA             2
CUFLA             2
NAIA DI           1
Name: division, dtype: int64

In [119]:
commitments_df

,name,commitment_year,state,high_school,club,position,division,college,gender,club_rank,weight,club_year,height,web_source,profile_link
0,Isabella Sabatino,2022,MD,Towson,SkyWalkers,M,D3,Pomona Pitzer,girl,NaN,NaN,NaN,NaN,laxnumbers,NaN
1,Natalie Clere,2022,NY,Onondaga,NaN,M,D2,Lindenwood,girl,NaN,NaN,NaN,NaN,laxnumbers,NaN
2,Claire Ashburn,2022,MD,Harford,NaN,A,D1,Winthrop,girl,NaN,NaN,NaN,NaN,laxnumbers,NaN
3,Gabby Trzeciak,2022,MD,Harford,Looneys,A,D2,Lock Haven,girl,NaN,NaN,NaN,NaN,laxnumbers,NaN
4,Trysten Burns,2022,NY,Onondaga,Haudenosaunee,A,D2,Newberry,girl,NaN,NaN,NaN,NaN,laxnumbers,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49753,Zoe Stewart,2025,NaN,Manatee,TRUE,D,NaN,Lynn,girl,NaN,NaN,NaN,NaN,clublacrosse,NaN
49754,Zoe Stonecipher,2025,NaN,Fox Lane,Primetime,A,NaN,Stony Brook,girl,NaN,NaN,NaN,NaN,clublacrosse,NaN
49755,Zoey Smith,2025,NaN,Lacey Twp,DEWLAX,M,NaN,Johns Hopkins,girl,NaN,NaN,NaN,NaN,clublacrosse,NaN
49756,Zola LeDonne,2025,NaN,Other,TEAM BC,A,NaN,Detroit Mercy,girl,NaN,NaN,NaN,NaN,clublacrosse,NaN


#### 02. Summary table for commitments

In [27]:
# create commitments count based on columns

In [28]:
# group_by_columns = ['club','division','gender','commitment_year','state']
# summary_df=commitments_df[['club','division','gender','commitment_year','state']].copy()
# summary_df

,club,division,gender,commitment_year,state
0,SkyWalkers,D3,girl,2022,MD
1,NaN,D2,girl,2022,NY
2,NaN,D1,girl,2022,MD
3,Looneys,D2,girl,2022,MD
4,Haudenosaunee,D2,girl,2022,NY
...,...,...,...,...,...
49753,TRUE,NaN,girl,2025,NaN
49754,Primetime,NaN,girl,2025,NaN
49755,DEWLAX,NaN,girl,2025,NaN
49756,TEAM BC,NaN,girl,2025,NaN


In [29]:
# summary_df.insert(0,'id',range(0,len(summary_df)))

In [32]:
# summary_table = summary_df.groupby(group_by_columns)['id'].agg('size').reset_index()
# summary_table=summary_table.rename(columns={'id':'commitments'})
# summary_table

,club,division,gender,commitment_year,state,commitments
0,02,D1,boy,2023,NJ,1
1,02/Nation United,D1,boy,2021,NJ,1
2,1,Club,boy,2021,UT,1
3,1,D1,boy,2020,FL,2
4,1,D1,boy,2021,IL,1
...,...,...,...,...,...,...
15515,t3/Dynamite,D2,girl,2021,PA,1
15516,t3/FCA,D1,girl,2020,PA,1
15517,t3/Jersey Shore,D3,girl,2021,NJ,2
15518,t3/SJ Select,D3,girl,2020,NJ,1


In [ ]:
# summary_df.groupby(['club','division','gender','commitment_year','state']).agg({'id':['count']})

In [ ]:
# grouped_df=summary_df.groupby(['club','division','gender','commitment_year','state'])
# group_sizes = grouped_df.size()
# print(group_sizes)

In [ ]:
# group_sizes_df=pd.DataFrame(group_sizes)
# # group_sizes_df

In [ ]:
# print(grouped_df.groups)

#####  Division column processing

In [ ]:
commitments_df['division'].isna().value_counts()

False    43649
True      5820
Name: division, dtype: int64

##### position processing

In [ ]:
commitments_df['position'].isna().value_counts()

False    48626
True       843
Name: position, dtype: int64

In [35]:
commitments_df['position'].value_counts()

M                                                           12397
A                                                            9487
D                                                            8963
G                                                            3619
Attack                                                       3066
Defense                                                      2608
Midfield                                                     2026
Middie                                                       1605
LSM                                                          1353
Goalie                                                       1183
FO                                                            985
Midflied                                                      422
Long Stick Midfield|Midfield|Defense|Face Off Specialist      178
FO/Draw                                                       156
Face Off Specialist                                           132
Face Off S

In [36]:
df_postion = commitments_df.copy()

In [37]:
df_postion['position'].isna().value_counts()

False    48626
True       843
Name: position, dtype: int64

In [42]:
input_string = 'Midfield'
position_map_dict = {
    'Midfield':'M',
    'Middie':'M',
    'Midflied':'M',
    'Midfiled':'M',
    'Defense':'D',
    'Attack':'A',
    'Goal':'G',
    'SSMF':'M',
    'FO/DRAW':'FO',
    'Face Off Specialist':'FO',
    'FOGO':'FO',
    'Faceoff':'FO',
    'Draw':'DRAW'
    }

In [44]:
df_postion['position']=df_postion['position'].apply(apply_position_map)

In [45]:
df_postion['position'].value_counts()

M       16953
A       12584
D       11594
G        4809
LSM      1353
FO       1325
DRAW        8
Name: position, dtype: int64

In [46]:
df_postion['position'].isna().value_counts()

False    48626
True       843
Name: position, dtype: int64

In [ ]:
# for rest of cases, we either remove those or replace with nearest options


#### Camps And Clinics

In [ ]:
camps_clinics_file_path = '../data/camps_and_clinics/gold/camps_and_clinics.csv'

In [ ]:
camps_df = pd.read_csv(camps_clinics_file_path)

In [ ]:
camps_df

##### fix formating for event_date

In [ ]:
from datetime import datetime
import numpy as np

In [ ]:
def process_date_format(input_date:str):
    if input_date is None:
        return None
    if type(input_date) == float:
        return input_date
    else:
        return datetime.date(input_date).strftime('%m/%d/%Y')

In [ ]:
camps_df['event_date']=pd.to_datetime(camps_df['event_date']).map(process_date_format)
camps_df

In [ ]:
camps_df['added']=pd.to_datetime(camps_df['added']).map(lambda x : process_date_format(x) if type(x)==pd.Timestamp else None)
camps_df

#### clubs Ranking data processing

In [ ]:
clubs_ranking_file_path = '../data/club_ranking/gold/club_ranking.csv'
clubs_ranking_df = pd.read_csv(clubs_ranking_file_path)

In [ ]:
clubs_ranking_df

In [ ]:
clubs_ranking_df['rank'][0]

In [ ]:
type(clubs_ranking_df['rank'][0])

In [ ]:
clubs_ranking_df['rank'].value_counts()

In [ ]:
clubs_ranking_df['rank'].map(lambda x: None if x=='--' else x).value_counts()

In [ ]:
clubs_ranking_df['rating'].value_counts()

In [ ]:
clubs_ranking_df['rating'].map(lambda x: None if x=='--' else x).value_counts()

#### school ranking data

In [ ]:
school_ranking_file_path = '../data/school_ranking/silver/laxmath.csv'
school_ranking_df = pd.read_csv(school_ranking_file_path)

In [ ]:
school_ranking_df

In [ ]:
school_ranking_df['team'].map(lambda x: x.split('(')[0] if type(x)==str else x)

In [ ]:
school_ranking_df['winRatio']= school_ranking_df['team'].map(lambda x: x.split('( ')[1].split(' )')[0] if (type(x)==str and '(' in x) else None)
school_ranking_df

In [ ]:
school_ranking_df['team']=school_ranking_df['team'].map(lambda x: x.split('( ')[0].split(' )')[0] if (type(x)==str and '(' in x) else None)
school_ranking_df

In [ ]:
school_ranking_df['champ_%']=school_ranking_df['champ_%'].map(lambda y: float(y.split('%')[0].replace(' ','')) if type(y)==str else y)
school_ranking_df

In [ ]:
school_ranking_df['champ_%'].value_counts()

#### College Rankings

In [ ]:
college_ranking_file_path = '../data/college_ranking/silver/laxmath.csv'
college_ranking_df = pd.read_csv(college_ranking_file_path)

In [ ]:
college_ranking_df

In [ ]:
college_ranking_df['winRatio']= college_ranking_df['team'].map(lambda x: x.split('( ')[1].split(' )')[0] if (type(x)==str and '(' in x) else None)
college_ranking_df
# for x,y in college_ranking_df['team'].map(lambda x: x.split('(')[1].split(')')[0] if type(x)==str else x).value_counts().items():
#     print(x)
#     print(y)


In [ ]:
college_ranking_df['winRatio'].value_counts()

In [ ]:
college_ranking_df['team']=college_ranking_df['team'].map(lambda x: x.split('(')[0] if type(x)==str else x)
college_ranking_df

In [ ]:
college_ranking_df['champ_%']=college_ranking_df['champ_%'].map(lambda y: float(y.split('%')[0].replace(' ','')) if type(y)==str else y)
college_ranking_df

In [ ]:
college_ranking_df.columns

In [ ]:
# update column maps
columns_map = {
    'champ_%_rank':'championshipPercentageRank',
    'pr':'powerRating',
    'sos-pr':'strengthOfSchedulePR',
    'qwf-pr':'qualityWinFactorPR',
    'champ_%':'championshipPercentage',
    'rpi':'ratingPercentageIndex',
    'sos-rpi':'strengthOfScheduleRPI',
    'qwf-rpi':'qualityWinFactorRPI',
    'selection':'selection',
    'champion_%':'championPercentage',
    }

In [ ]:
from data_helper import map_col_names

In [ ]:
college_ranking_df=map_col_names(college_ranking_df,columns_map)
college_ranking_df.columns